In [97]:
import pandas as pd
import numpy as np
import random

excel_file = '1017168_coursetimemajor_2018-04-23.xlsx'
schedule = pd.read_excel(excel_file)

In [103]:
#-------------------------------------------------------------
# verify or change the following value as you import a new file
#-------------------------------------------------------------

#the column that contains days of which courses are offering on
day_col = 'Days_of_week'

#the column that identifies student
stu_col = 'Key'

year = 'Class'

Start_time = 'Start_time'

End_time = 'End_time'

quarter = np.int64(20172)


#headers
cols = list(schedule)
cols.remove(day_col)


#---------
#a function that outputs course and student's information based on the row index associated with the day

#input: the row index of the current day; keep track of student's info
#output: a list of student's info except the Days_of_week info
#@staticmethod
def outputOtherInfo(row_idx, df):
    otherInfoList = []
    
    for col in cols:
        #if not col == "Days_of_week":
        otherInfoList.append(df[col][row_idx])
    return otherInfoList

In [100]:
#------------------------------------------------
# Filter out pre major freshmen and sophomore
#------------------------------------------------

quarter_schedule = schedule.loc[schedule['YrQtr'] == quarter]

freshmen = quarter_schedule.loc[quarter_schedule[year] == np.int64(1)]
sophomore = quarter_schedule.loc[quarter_schedule[year] == np.int64(2)]

premajor = [freshmen, sophomore]

In [101]:
#------------------
# random sampling for each class now is freshmen or change freshmen to sophomore
#------------------

sample_table = []

unique_list = []

# function to get unique values
def unique(students):
    # traverse for all elements
    for x in students:
        
        # check if exists in unique_list or not
        if x not in unique_list:
            unique_list.append(x)

#--------------------------
#first get a unique list of students key
for each_class in premajor:
    
    unique_list = []
    
    unique(each_class[stu_col])
    
    #--------------------------------------
    #then do random sampling within the list
    
    sample_students = []
    
    sample_students = random.sample(unique_list, int(len(unique_list)/ 2))

    #------------------------------------------------------
    #then selecting the rows that contain sampled students key

    for student in sample_students:
        something = each_class.loc[each_class[stu_col] == student]
        sample_table.append(something)
        
output_table = pd.concat(sample_table)
final_table = output_table.reset_index(drop=True)

In [ ]:
#-----------------------
# separating days column
#-----------------------

week = final_table[day_col]

#create a new table to store separated courses' information
new_Table = []

new_headers = ['Day'] + cols


#loop through the days that are in each cell
#combine corresponding class and student info into a new list
#append the list into the new table

for idx, days in enumerate(week):
    for day in days:
        if not day.isspace():
            
            #create a new list for each day
            each_day = []
            each_day.append(day)
            each_day = each_day + outputOtherInfo(idx, final_table)
            new_Table.append(each_day)

#-------
#output the new Table into a dataframe with columns 
new_schedule = pd.DataFrame(new_Table, columns = list(new_headers))
new_schedule

#----------
#sort new_schedule by Student_ID 1, then by Day 0, then by Pm, finally by start time 7
sorted_schedule = new_schedule.sort_values(by=[stu_col, 'Day', Start_time])
new_schedule = sorted_schedule.reset_index(drop=True)

In [ ]:
#---------------------------------
# take out non-consecutive classes 
#---------------------------------

cons_class_table = []
cols = list(new_schedule)
timegapList = [0]

#verify if the student id and the day are the same
#return the time gap between classes if the same
# return 999 as time gap if not the same
def gapTimeWithNextClass(clas1, idx):
    new_idx = idx + 1 
    timegap = 0
    
    #next class
    clas2 = [new_schedule[stu_col][new_idx], new_schedule['Day'][new_idx], new_schedule[Start_time][new_idx]]
    
    if (clas1[0] == clas2[0] and clas1[1] == clas2[1]):
        timegap = clas2[2] - clas1[2]
    else:
        timegap = 999
    
    timegapList.append(timegap)
    
    return timegap        
    
#
def create_cons_class_table(clas1_idx):
    clas2_idx = clas1_idx + 1
    
    clas1_info = outputOtherInfo(i, new_schedule)
    clas2_info = outputOtherInfo(clas2_idx, new_schedule)

    cons_class_table.append(clas1_info)
    cons_class_table.append(clas2_info)

    
for i in range(new_schedule.shape[0]-1):
    class1 = [new_schedule[stu_col][i], new_schedule['Day'][i], new_schedule[End_time][i]]
    
    time = gapTimeWithNextClass(class1, i)
        
    # if there is 10 mins gap in between classes
    if (time == 10 or time == 50):
        create_cons_class_table(i)
    
new_schedule['TimeGap'] = timegapList

cons_schedule = pd.DataFrame(cons_class_table, columns = list(new_headers))

In [ ]:
cons_schedule.to_csv('time_schedule.csv')